In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from collections import Counter
from itertools import chain
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import seaborn as sns
from wordcloud import WordCloud

In [2]:
file_path = "cleaned.csv"  # Update with actual file path

df = pd.read_csv(file_path)


In [3]:
df.head

<bound method NDFrame.head of          Unnamed: 0  rating  \
0                 0     5.0   
1                 1     5.0   
2                 2     2.0   
3                 3     1.0   
4                 4     5.0   
...             ...     ...   
2500934     2500934     5.0   
2500935     2500935     1.0   
2500936     2500936     3.0   
2500937     2500937     5.0   
2500938     2500938     1.0   

                                                     title  \
0                                            Pretty locket   
1                                                        A   
2                                                Two Stars   
3                                          Won’t buy again   
4                                     I LOVE these glasses   
...                                                    ...   
2500934  ... allowed them to be used to add military ri...   
2500935                           Didn’t come with all ten   
2500936                            Not 

In [4]:
# === STEP 1: Preprocess and Label ===
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Label each review with lockdown period
df['lockdown_period'] = df['timestamp'].apply(
    lambda x: 'Post-COVID' if x >= pd.Timestamp('2020-03-15') else 'Pre-COVID'
)

def get_season(month):
    if month in [4, 5, 6, 7, 8]:
        return 'Summer'
    elif month in [11, 12, 1, 2]:
        return 'Winter'
    else:
        return 'Other'

df['season'] = df['timestamp'].dt.month.apply(get_season)
df = df[df['season'].isin(['Summer', 'Winter'])]

In [5]:

# Generalize Rating
rating_hierarchy = {
    1: 'Low',
    2: 'Low',
    3: 'Medium',
    4: 'High',
    5: 'High'
}
df['rating_generalized'] = df['rating'].map(rating_hierarchy)

# Add Sentiment based on rating
def map_sentiment(rating):
    if rating >= 4:
        return 'Positive'
    elif rating == 3:
        return 'Neutral'
    else:
        return 'Negative'

df['sentiment'] = df['rating'].apply(map_sentiment)

# Generalize Review Length
def categorize_length(length):
    if length < 50:
        return 'Short'
    elif 50 <= length <= 100:
        return 'Medium'
    else:
        return 'Long'

df['review_length_category'] = df['cleaned_text'].apply(
    lambda x: categorize_length(len(str(x).split()))
)

# Generalize Helpful Votes
def generalize_helpful(votes):
    if votes == 0:
        return 'None'
    elif 1 <= votes <= 5:
        return 'Less Helpful'
    else:
        return 'More Helpful'

df['helpful_vote_category'] = df['helpful_vote'].apply(generalize_helpful)

# Generalize Verified Purchase
df['verified_status'] = df['verified_purchase'].map({True: 'Verified', False: 'Not Verified'})



# Final selection of generalized attributes
generalized_df = df[[
    'rating_generalized',
    'sentiment',
    'review_length_category',
    'helpful_vote_category',
    'verified_status',
    'lockdown_period',
    'season'
]]

# Display top rows
print(generalized_df.head())


C:\Users\krish\AppData\Local\Temp\ipykernel_9300\545175802.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating_generalized'] = df['rating'].map(rating_hierarchy)
C:\Users\krish\AppData\Local\Temp\ipykernel_9300\545175802.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['rating'].apply(map_sentiment)
C:\Users\krish\AppData\Local\Temp\ipykernel_9300\545175802.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

  rating_generalized sentiment review_length_category helpful_vote_category  \
0               High  Positive                  Short          Less Helpful   
1               High  Positive                  Short                  None   
2                Low  Negative                  Short          Less Helpful   
3                Low  Negative                  Short          Less Helpful   
4               High  Positive                  Short                  None   

  verified_status lockdown_period  season  
0        Verified       Pre-COVID  Winter  
1        Verified      Post-COVID  Winter  
2        Verified       Pre-COVID  Summer  
3        Verified       Pre-COVID  Winter  
4        Verified       Pre-COVID  Summer  


C:\Users\krish\AppData\Local\Temp\ipykernel_9300\545175802.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['verified_status'] = df['verified_purchase'].map({True: 'Verified', False: 'Not Verified'})


In [6]:
summary = df.groupby(
    ["lockdown_period", "season", "rating_generalized", "sentiment", 
     "review_length_category", "helpful_vote_category", "verified_status"]
).agg(
    review_count=("rating", "count"),
    avg_rating=("rating", "mean"),
    avg_helpful_vote=("helpful_vote", "mean")
).reset_index()


In [7]:
summary

,lockdown_period,season,rating_generalized,sentiment,review_length_category,helpful_vote_category,verified_status,review_count,avg_rating,avg_helpful_vote
0,Post-COVID,Summer,High,Positive,Long,Less Helpful,Not Verified,223,4.578475,1.798206
1,Post-COVID,Summer,High,Positive,Long,Less Helpful,Verified,311,4.710611,2.234727
2,Post-COVID,Summer,High,Positive,Long,More Helpful,Not Verified,15,4.733333,8.733333
3,Post-COVID,Summer,High,Positive,Long,More Helpful,Verified,149,4.691275,21.604027
4,Post-COVID,Summer,High,Positive,Long,None,Not Verified,330,4.639394,0.000000
...,...,...,...,...,...,...,...,...,...,...
210,Pre-COVID,Winter,Medium,Neutral,Short,Less Helpful,Verified,10688,3.000000,1.576254
211,Pre-COVID,Winter,Medium,Neutral,Short,More Helpful,Not Verified,34,3.000000,11.588235
212,Pre-COVID,Winter,Medium,Neutral,Short,More Helpful,Verified,630,3.000000,11.634921
213,Pre-COVID,Winter,Medium,Neutral,Short,None,Not Verified,2352,3.000000,0.000000


In [8]:
# Convert the summary dataframe into a transaction-style format
summary_trans = df.apply(lambda row: [
    f"lockdown_period={row['lockdown_period']}",
    f"season={row['season']}",
    f"Rating={row['rating_generalized']}",
    f"Sentiment={row['sentiment']}",
    f"Review_Length={row['review_length_category']}",
    f"Helpfulness={row['helpful_vote_category']}",
    f"Verified={row['verified_status']}"
], axis=1)

# Convert the list of transactions to the right format for rule mining
summary_trans = summary_trans.tolist()

# Example: Print the first 5 transactions
print("Sample Transactions (first 5):", summary_trans[:5])


Sample Transactions (first 5): [['lockdown_period=Pre-COVID', 'season=Winter', 'Rating=High', 'Sentiment=Positive', 'Review_Length=Short', 'Helpfulness=Less Helpful', 'Verified=Verified'], ['lockdown_period=Post-COVID', 'season=Winter', 'Rating=High', 'Sentiment=Positive', 'Review_Length=Short', 'Helpfulness=None', 'Verified=Verified'], ['lockdown_period=Pre-COVID', 'season=Summer', 'Rating=Low', 'Sentiment=Negative', 'Review_Length=Short', 'Helpfulness=Less Helpful', 'Verified=Verified'], ['lockdown_period=Pre-COVID', 'season=Winter', 'Rating=Low', 'Sentiment=Negative', 'Review_Length=Short', 'Helpfulness=Less Helpful', 'Verified=Verified'], ['lockdown_period=Pre-COVID', 'season=Summer', 'Rating=High', 'Sentiment=Positive', 'Review_Length=Short', 'Helpfulness=None', 'Verified=Verified']]


In [9]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

# Create a TransactionEncoder object to convert the transaction data
te = TransactionEncoder()
te_ary = te.fit(summary_trans).transform(summary_trans)
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

In [10]:
# Apply apriori with a lower support threshold (e.g., 0.05)
frequent_itemsets = apriori(df_trans, min_support=0.05, use_colnames=True)

# Generate association rules based on confidence with a minimum threshold of 0.2
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

# === Step 1: Filter for 'lockdown_period=Post-COVID' in antecedents ===
post_covid_rules = rules[rules['antecedents'].apply(
    lambda x: 'lockdown_period=Post-COVID' in str(x) and len(x) == 1)]  # Strict filter for 'Post-COVID' alone

# === Step 2: Filter for 'lockdown_period=Pre-COVID' in antecedents ===
pre_covid_rules = rules[rules['antecedents'].apply(
    lambda x: 'lockdown_period=Pre-COVID' in str(x) and len(x) == 1)]  # Strict filter for 'Pre-COVID' alone

# === Step 3: Sort the rules by lift in descending order ===
post_covid_rules = post_covid_rules.sort_values(by='lift', ascending=False)
pre_covid_rules = pre_covid_rules.sort_values(by='lift', ascending=False)

# === Step 4: Display Rules for Post-COVID ===
print("\n📋 Lockdown Period-Based Discrimination Rules (Strictly 'lockdown_period=Post-COVID'):\n")
if not post_covid_rules.empty:
    for _, row in post_covid_rules.iterrows():
        antecedent = ' AND '.join(list(row['antecedents']))
        consequent = ' AND '.join(list(row['consequents']))
        print(f"-> IF {antecedent} THEN {consequent}")
        print(f"   - Support: {round(row['support'], 3)}")
        print(f"   - Confidence: {round(row['confidence'], 3)}")
        print(f"   - Lift: {round(row['lift'], 3)}\n")
else:
    print("No rules generated for 'lockdown_period=Post-COVID'.")

# === Step 5: Display Rules for Pre-COVID ===
print("\n📋 Lockdown Period-Based Discrimination Rules (Strictly 'lockdown_period=Pre-COVID'):\n")
if not pre_covid_rules.empty:
    for _, row in pre_covid_rules.iterrows():
        antecedent = ' AND '.join(list(row['antecedents']))
        consequent = ' AND '.join(list(row['consequents']))
        print(f"-> IF {antecedent} THEN {consequent}")
        print(f"   - Support: {round(row['support'], 3)}")
        print(f"   - Confidence: {round(row['confidence'], 3)}")
        print(f"   - Lift: {round(row['lift'], 3)}\n")
else:
    print("No rules generated for 'lockdown_period=Pre-COVID'.")

# === Step 6: Summary Tables ===
# Create summary tables for each filter
post_covid_summary_df = post_covid_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].copy()
post_covid_summary_df['antecedents'] = post_covid_summary_df['antecedents'].apply(lambda x: ' AND '.join(x))
post_covid_summary_df['consequents'] = post_covid_summary_df['consequents'].apply(lambda x: ' AND '.join(x))

pre_covid_summary_df = pre_covid_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].copy()
pre_covid_summary_df['antecedents'] = pre_covid_summary_df['antecedents'].apply(lambda x: ' AND '.join(x))
pre_covid_summary_df['consequents'] = pre_covid_summary_df['consequents'].apply(lambda x: ' AND '.join(x))

# Save the summaries to separate CSV files
post_covid_summary_df.to_csv('lockdown_period_post_covid_rules_summary.csv', index=False)
pre_covid_summary_df.to_csv('lockdown_period_pre_covid_rules_summary.csv', index=False)

# Print the summary of the rules
print("\n📄 Summary of Lockdown Period Discrimination Rules (Post-COVID):\n")
print(post_covid_summary_df.head(10))

print("\n📄 Summary of Lockdown Period Discrimination Rules (Pre-COVID):\n")
print(pre_covid_summary_df.head(10))



📋 Lockdown Period-Based Discrimination Rules (Strictly 'lockdown_period=Post-COVID'):

-> IF lockdown_period=Post-COVID THEN Sentiment=Negative AND Review_Length=Short
   - Support: 0.064
   - Confidence: 0.218
   - Lift: 1.196

-> IF lockdown_period=Post-COVID THEN Review_Length=Short AND Rating=Low
   - Support: 0.064
   - Confidence: 0.218
   - Lift: 1.196

-> IF lockdown_period=Post-COVID THEN Review_Length=Short AND Sentiment=Negative AND Rating=Low
   - Support: 0.064
   - Confidence: 0.218
   - Lift: 1.196

-> IF lockdown_period=Post-COVID THEN Sentiment=Negative
   - Support: 0.066
   - Confidence: 0.227
   - Lift: 1.194

-> IF lockdown_period=Post-COVID THEN Sentiment=Negative AND Rating=Low
   - Support: 0.066
   - Confidence: 0.227
   - Lift: 1.194

-> IF lockdown_period=Post-COVID THEN Rating=Low
   - Support: 0.066
   - Confidence: 0.227
   - Lift: 1.194

-> IF lockdown_period=Post-COVID THEN Review_Length=Short AND Verified=Verified AND Rating=Low
   - Support: 0.059
   

In [11]:
import os

# Re-run Apriori with lower thresholds to capture more rules
frequent_itemsets = apriori(df_trans, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

# Create output directory
os.makedirs("season_rules", exist_ok=True)

# Define seasons
seasons = ['Winter', 'Summer']

print("\n Generating Discrimination Rules Based on Season \n")

for season in seasons:
    season_tag = f"season={season}"
    
    # Strict filter: only that season alone in antecedent
    season_rules = rules[rules['antecedents'].apply(
        lambda x: season_tag in str(x) and len(x) == 1
    )].sort_values(by='lift', ascending=False)

    print(f"\n Rules for '{season_tag}':")
    if not season_rules.empty:
        for _, row in season_rules.iterrows():
            antecedent = ' AND '.join(list(row['antecedents']))
            consequent = ' AND '.join(list(row['consequents']))
            print(f"-> IF {antecedent} THEN {consequent}")
            print(f"   - Support: {round(row['support'], 3)}")
            print(f"   - Confidence: {round(row['confidence'], 3)}")
            print(f"   - Lift: {round(row['lift'], 3)}\n")
        
        # Save summary to CSV
        summary_df = season_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].copy()
        summary_df['antecedents'] = summary_df['antecedents'].apply(lambda x: ' AND '.join(x))
        summary_df['consequents'] = summary_df['consequents'].apply(lambda x: ' AND '.join(x))
        summary_df.to_csv(f"season_rules/{season.lower()}_season_rules_summary.csv", index=False)
    else:
        print(f"❌ No rules generated for season '{season}'.")



 Generating Discrimination Rules Based on Season 


 Rules for 'season=Winter':
-> IF season=Winter THEN Sentiment=Positive AND lockdown_period=Pre-COVID AND Helpfulness=None AND Review_Length=Short
   - Support: 0.2
   - Confidence: 0.428
   - Lift: 1.068

-> IF season=Winter THEN Review_Length=Short AND Helpfulness=None AND Sentiment=Positive AND lockdown_period=Pre-COVID AND Rating=High
   - Support: 0.2
   - Confidence: 0.428
   - Lift: 1.068

-> IF season=Winter THEN Rating=High AND lockdown_period=Pre-COVID AND Helpfulness=None AND Review_Length=Short
   - Support: 0.2
   - Confidence: 0.428
   - Lift: 1.068

-> IF season=Winter THEN Review_Length=Short AND Helpfulness=None AND Verified=Verified AND lockdown_period=Pre-COVID AND Rating=High
   - Support: 0.19
   - Confidence: 0.408
   - Lift: 1.066

-> IF season=Winter THEN Review_Length=Short AND Helpfulness=None AND Verified=Verified AND Sentiment=Positive AND lockdown_period=Pre-COVID
   - Support: 0.19
   - Confidence: 0.408

In [12]:
df_trans

,Helpfulness=Less Helpful,Helpfulness=More Helpful,Helpfulness=None,Rating=High,Rating=Low,Rating=Medium,Review_Length=Long,Review_Length=Medium,Review_Length=Short,Sentiment=Negative,Sentiment=Neutral,Sentiment=Positive,Verified=Not Verified,Verified=Verified,lockdown_period=Post-COVID,lockdown_period=Pre-COVID,season=Summer,season=Winter
0,True,False,False,True,False,False,False,False,True,False,False,True,False,True,False,True,False,True
1,False,False,True,True,False,False,False,False,True,False,False,True,False,True,True,False,False,True
2,True,False,False,False,True,False,False,False,True,True,False,False,False,True,False,True,True,False
3,True,False,False,False,True,False,False,False,True,True,False,False,False,True,False,True,False,True
4,False,False,True,True,False,False,False,False,True,False,False,True,False,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907673,False,False,True,True,False,False,False,False,True,False,False,True,False,True,False,True,True,False
1907674,False,False,True,True,False,False,False,False,True,False,False,True,False,True,False,True,True,False
1907675,False,False,True,False,True,False,False,False,True,True,False,False,False,True,False,True,True,False
1907676,False,False,True,False,False,True,False,False,True,False,True,False,False,True,False,True,False,True


In [ ]:
# ---------------------------
# Helper function to calculate T and D weights
# ---------------------------
def calculate_weights_percentage(rules_df, season_column):
    def compute(row):
        in_antecedent = season_column in str(row['antecedents'])
        t_weight = row['support'] * 100 if in_antecedent else 0
        d_weight = row['lift'] * 100 if in_antecedent else 0
        return pd.Series([t_weight, d_weight])
    
    rules_df[['T Weight (%)', 'D Weight (%)']] = rules_df.apply(compute, axis=1)

    # Cap the D-Weight to not exceed 100% after computation
    rules_df['D Weight (%)'] = rules_df['D Weight (%)'].apply(lambda x: min(x, 100))

    return rules_df

# ---------------------------
# List of seasons (adapt based on your data)
# ---------------------------
seasons = ['Summer', 'Winter']

# ---------------------------
# Main loop to process each season
# ---------------------------
for season in seasons:
    season_tag = f"season={season}"

    if season_tag in df_trans.columns:
        # Filter rules where the season is the only antecedent
        season_rules = rules[rules['antecedents'].apply(
            lambda x: season_tag in str(x) and len(x) == 1
        )].sort_values(by='D weight', ascending=False)

        print(f"\n Rules for '{season_tag}':")
        if not season_rules.empty:
            # ✅ Add T/D Weights
            season_rules = calculate_weights_percentage(season_rules, season_tag)

            for _, row in season_rules.iterrows():
                antecedent = ' AND '.join(list(row['antecedents']))
                consequent = ' AND '.join(list(row['consequents']))
                print(f"-> IF {antecedent} THEN {consequent}")
                print(f"   - Support: {round(row['support'], 3)}")
                print(f"   - Confidence: {round(row['confidence'], 3)}")
                print(f"   - Lift: {round(row['lift'], 3)}")
                print(f"   - T Weight (Typicality): {row['T Weight (%)']:.2f}%")
                print(f"   - D Weight (Discriminability): {row['D Weight (%)']:.2f}%")

            # ✅ Save to CSV
            summary_df = season_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift',
                                       'T Weight (%)', 'D Weight (%)']].copy()
            summary_df['antecedents'] = summary_df['antecedents'].apply(lambda x: ' AND '.join(x))
            summary_df['consequents'] = summary_df['consequents'].apply(lambda x: ' AND '.join(x))
            summary_df.to_csv(f"season_rules/{season}_season_rules_summary.csv", index=False)
        else:
            print(f"❌ No rules generated for season '{season}'.")
    else:
        print(f"❌ Column '{season_tag}' not found in the DataFrame.")


 Rules for 'season=Summer':
-> IF season=Summer THEN Verified=Verified AND lockdown_period=Post-COVID
   - Support: 0.156
   - Confidence: 0.292
   - Lift: 1.095
   - T Weight (Typicality): 15.60%
   - D Weight (Discriminability): 100.00%
-> IF season=Summer THEN Verified=Verified AND lockdown_period=Post-COVID AND Review_Length=Short
   - Support: 0.151
   - Confidence: 0.282
   - Lift: 1.095
   - T Weight (Typicality): 15.07%
   - D Weight (Discriminability): 100.00%
-> IF season=Summer THEN Helpfulness=None AND Verified=Verified AND lockdown_period=Post-COVID
   - Support: 0.13
   - Confidence: 0.244
   - Lift: 1.094
   - T Weight (Typicality): 13.00%
   - D Weight (Discriminability): 100.00%
-> IF season=Summer THEN Helpfulness=None AND Verified=Verified AND lockdown_period=Post-COVID AND Review_Length=Short
   - Support: 0.127
   - Confidence: 0.238
   - Lift: 1.094
   - T Weight (Typicality): 12.72%
   - D Weight (Discriminability): 100.00%
-> IF season=Summer THEN lockdown_peri

In [19]:
def calculate_weights_percentage(rules_df, lockdown_period_column):
    def compute(row):
        in_antecedent = lockdown_period_column in str(row['antecedents'])
        t_weight = row['support'] * 100 if in_antecedent else 0
        d_weight = row['lift'] * 100 if in_antecedent else 0
        return pd.Series([t_weight, d_weight])
    
    rules_df[['T Weight (%)', 'D Weight (%)']] = rules_df.apply(compute, axis=1)

    # Cap the D-Weight to not exceed 100% after computation
    rules_df['D Weight (%)'] = rules_df['D Weight (%)'].apply(lambda x: min(x, 100))

    return rules_df


# ---------------------------
# List of lockdown periods
# ---------------------------
lockdown_periods = ['Pre-COVID', 'Post-COVID']

# ---------------------------
# Main loop to process each lockdown period
# ---------------------------
for period in lockdown_periods:
    period_tag = f"lockdown_period={period}"

    if period_tag in df_trans.columns:
        # Filter rules where the lockdown period is the only antecedent
        period_rules = rules[rules['antecedents'].apply(
            lambda x: period_tag in str(x) and len(x) == 1
        )].sort_values(by='lift', ascending=False)

        print(f"\n Rules for '{period_tag}':")
        if not period_rules.empty:
            # ✅ Add T/D Weights
            period_rules = calculate_weights_percentage(period_rules, period_tag)

            for _, row in period_rules.iterrows():
                antecedent = ' AND '.join(list(row['antecedents']))
                consequent = ' AND '.join(list(row['consequents']))
                print(f"-> IF {antecedent} THEN {consequent}")
                print(f"   - Support: {round(row['support'], 3)}")
                print(f"   - Confidence: {round(row['confidence'], 3)}")
                print(f"   - Lift: {round(row['lift'], 3)}")
                print(f"   - T Weight (Typicality): {row['T Weight (%)']:.2f}%")
                print(f"   - D Weight (Discriminability): {row['D Weight (%)']:.2f}%")

            # ✅ Save to CSV
            summary_df = period_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift',
                                       'T Weight (%)', 'D Weight (%)']].copy()
            summary_df['antecedents'] = summary_df['antecedents'].apply(lambda x: ' AND '.join(x))
            summary_df['consequents'] = summary_df['consequents'].apply(lambda x: ' AND '.join(x))
            summary_df.to_csv(f"lockdown_period_rules/{period}_lockdown_period_rules_summary.csv", index=False)
        else:
            print(f"❌ No rules generated for lockdown period '{period}'.")
    else:
        print(f"❌ Column '{period_tag}' not found in the DataFrame.")
        print(f"❌ Column '{season_tag}' not found in the DataFrame.")


 Rules for 'lockdown_period=Pre-COVID':
-> IF lockdown_period=Pre-COVID THEN season=Winter AND Sentiment=Positive AND Verified=Verified AND Rating=High
   - Support: 0.239
   - Confidence: 0.338
   - Lift: 1.073
   - T Weight (Typicality): 23.89%
   - D Weight (Discriminability): 100.00%
-> IF lockdown_period=Pre-COVID THEN season=Winter AND Verified=Verified AND Rating=High
   - Support: 0.239
   - Confidence: 0.338
   - Lift: 1.073
   - T Weight (Typicality): 23.89%
   - D Weight (Discriminability): 100.00%
-> IF lockdown_period=Pre-COVID THEN season=Winter AND Sentiment=Positive AND Verified=Verified
   - Support: 0.239
   - Confidence: 0.338
   - Lift: 1.073
   - T Weight (Typicality): 23.89%
   - D Weight (Discriminability): 100.00%
-> IF lockdown_period=Pre-COVID THEN Rating=High AND season=Winter AND Verified=Verified AND Review_Length=Short
   - Support: 0.231
   - Confidence: 0.327
   - Lift: 1.073
   - T Weight (Typicality): 23.14%
   - D Weight (Discriminability): 100.00%
-